# PyTorch Redes Neurais Profundas - Exemplo

## Classificador de cães e gatos

Dada uma imagem que tem um cão ou gato, vamos ser capazes de dizer qual categoria ela está.

In [ ]:
import math
import matplotlib.pyplot as plt

%load_ext autoreload
%autoreload 2
%matplotlib inline

import time
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from torchvision import models
import numpy as np

from dataset_aula import MyDataset   # Classe que carrega dataset


In [ ]:
DEVICE = 'gpu'      # 'cpu'
GPU_NUMBER = 0

PREFIX = '/media/dpetrini/KINGSTON/Daniel/nova/data_cats_dogs_old'
NUM_EPOCHS = 30
MINI_BATCH = 4
LR = 3e-4            # Learning rate - Taxa de aprendizado
PRE_TRAINED = False
NUM_WORKERS = 2

Vamos criar uma rede com duas camadas convolucionais (feature extraction) e 2 camadas lineares (classificadores).

![title](images/RedeConvolucional.png)

In [ ]:
class RedeConv(nn.Module):
    """ Rede Convolucional de duas camadas """
    def __init__(self):
        super(RedeConv, self).__init__()

        self.feature_extractor = nn.Sequential(  # Rede Convolucional para extrair features
            nn.Conv2d(in_channels=3, out_channels=16, kernel_size=3, stride=1, padding=0),
            nn.MaxPool2d(kernel_size=2), 
            nn.ReLU(inplace=True),          # função de ativação não linear
            nn.Conv2d(in_channels=16, out_channels=32, kernel_size=3, stride=1, padding=0),
            nn.MaxPool2d(kernel_size=2),
            nn.ReLU(inplace=True),
            )
        
        self.classifier = nn.Sequential(         # Rede Neural Artificial para classificar
            nn.Linear(32*54*54, 100),
            nn.Linear(100, 2)
        )

    def forward(self, x):

        x = self.feature_extractor(x)
        x = torch.flatten(x, 1)
        x = self.classifier(x)
        return x

In [ ]:
# Funcao que executa treino e validação por diversas épocas.

def train_and_validate(model, loss_criterion, optimizer, 
                       train_data_loader, validation_data_loader, 
                       device, epochs=25, batch_size=1):

    train_data_size = len(train_data_loader) * batch_size
    val_data_size = len(validation_data_loader) * batch_size

    for epoch in range(epochs):
        epoch_start = time.time()
        print('Epoch: {}/{}'.format(epoch+1, epochs))

        model.train()       # set to training mode

        #loss and acc for the epoch
        train_loss, train_acc = 0.0, 0.0
        validation_loss, validation_acc = 0.0, 0.0

        for i, (inputs, labels) in enumerate(train_data_loader):  # no iterator com tam=bs

            inputs = inputs.to(device)
            labels = labels.to(device)       # Aprendizagem supervisionada - ja tenho os resultados do treino

            optimizer.zero_grad()                       # clean existing gradients
            outputs = model(inputs)                     # forward pass
            loss = loss_criterion(outputs, labels)      # compute loss
            loss.backward()                             # backprop the gradients
            optimizer.step()                            # update parameters
            train_loss += loss.item() * inputs.size(0)  # compute the total loss for the batch & add

            # compute the accuracy
            acc = (torch.argmax(outputs, dim=1) == labels).float().sum()

            # compute total accuracy in the whole batch and add to train_acc
            train_acc += acc.item()

        # validation - no gradient tracking needed
        with torch.no_grad():

            model.eval()        # set to evaluation mode

            # validation loop
            for j, (inputs, labels) in enumerate(validation_data_loader):
                inputs = inputs.to(device)
                labels = labels.to(device)

                outputs = model(inputs)                         # forward pass for validation
                loss = loss_criterion(outputs, labels)	        # compute loss
                validation_loss += loss.item() * inputs.size(0) # compute the total loss for the batch & add

                # calculcate validation acc
                acc = (torch.argmax(outputs, dim=1) == labels).float().sum()

                # compute total accuracy in the whole batch and addd to valid_acc
                validation_acc += acc.item()

        # fing average training loss and training accuracy
        avg_train_loss = train_loss/train_data_size
        avg_train_acc = train_acc/train_data_size

        # find average training loss and validation acc
        avg_validation_loss = validation_loss/val_data_size
        avg_validation_acc = validation_acc/val_data_size

        epoch_end = time.time()

        print('Epoch: {:03d}, Training: Loss: {:.4f}, Accuracy: {:.2f}%, Validation: Loss: {:0.4f}, Accuracy: {:.2f}%, Time: {:.2f}s'.format(epoch+1, avg_train_loss, avg_train_acc*100, avg_validation_loss, avg_validation_acc*100, epoch_end-epoch_start))

In [ ]:
def initialize(model):
    """ good init for not pre-trained """
    print("Init weights with kaiming")
    for m in model.modules():
        if isinstance(m, nn.Conv2d):
            nn.init.kaiming_normal_(m.weight)
        elif isinstance(m, (nn.BatchNorm2d, nn.GroupNorm)):
            nn.init.constant_(m.weight, 1)
            nn.init.constant_(m.bias, 0)


In [ ]:
model = RedeConv()

print(model)

In [ ]:
initialize(model)


if (DEVICE == "gpu") and torch.has_cudnn:
    device = torch.device("cuda:{}".format(GPU_NUMBER))
else:
    device = torch.device("cpu")

model = model.to(device)

In [ ]:
train_image_paths = PREFIX+'/train'
val_image_paths = PREFIX+'/val'
test_image_paths = PREFIX+'/test'

# Carrega datasets
dataset_train = MyDataset(train_image_paths, train=True)
dataset_val = MyDataset(val_image_paths, train=False)
dataset_test = MyDataset(test_image_paths, train=False)

train_dataloader = DataLoader(dataset_train, batch_size=MINI_BATCH, shuffle=True)
val_dataloader = DataLoader(dataset_val, batch_size=MINI_BATCH, shuffle=False)
test_dataloader = DataLoader(dataset_test, batch_size=MINI_BATCH, shuffle=False)

print('\nSize train:', len(dataset_train), ' Size val: ', len(dataset_val))

In [ ]:
#Mostrar exemplos do dataset
def show_image(im, label, ax=None, figsize=(3, 3)):
    if ax is None: _, ax = plt.subplots(1, 1, figsize=figsize)
    ax.axis('off')
    ax.set_title(dataset_train.get_category(label))
    #print(im.shape, im.type())
    im = im.permute(1, 2, 0)
    ax.imshow(im)


def show_batch(x, label, c=4, r=None, figsize=None):
    n = len(x)
    cont = 0
    if r is None: r = int(math.ceil(n/c))
    if figsize is None: figsize = (c*3, r*3)
    fig, axes = plt.subplots(r, c, figsize=figsize)
    for xi, ax in zip(x, axes.flat):
        show_image(xi, label[cont], ax)
        cont += 1
    plt.show()



In [ ]:
# show some samples
img, label = next(iter(train_dataloader))
show_batch(img[0:20], label, 4)

In [ ]:
loss_func = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=LR)

In [ ]:
train_and_validate(model, loss_func, optimizer, train_dataloader, val_dataloader, device,
                   epochs=NUM_EPOCHS, batch_size=MINI_BATCH)

## Resultados:
    RedeConv 1: 70.67%

Como podemos melhorar esse resultado?

Vamos criar uma rede mais profunda, com uma camada convolucional a mais.

![title](images/RedeConvolucionalMaisProfunda.png)

In [ ]:
class RedeConvMaisProfunda(nn.Module):
    """ Rede com uma camada convolucional a mais """
    def __init__(self):
        super(RedeConvMaisProfunda, self).__init__()

        self.feature_extractor = nn.Sequential(
            nn.Conv2d(in_channels=3, out_channels=16, kernel_size=3, stride=1, padding=0),
            nn.MaxPool2d(kernel_size=2),
            nn.ReLU(inplace=True),
            nn.Conv2d(in_channels=16, out_channels=32, kernel_size=3, stride=1, padding=0),
            nn.MaxPool2d(kernel_size=2),
            nn.ReLU(inplace=True),
            nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, stride=1, padding=0),
            nn.MaxPool2d(kernel_size=2),
            nn.ReLU(inplace=True)                        
            )

        self.classifier = nn.Sequential(
            nn.Linear(64*26*26, 100),
            nn.Linear(100, 2)
        )

    def forward(self, x):
        x = self.feature_extractor(x)
        x = torch.flatten(x, 1)
        x = self.classifier(x)
        return x

In [ ]:
model2 = RedeConvMaisProfunda()

print(model2)

In [ ]:
initialize(model2)


if (DEVICE == "gpu") and torch.has_cudnn:
    device = torch.device("cuda:{}".format(GPU_NUMBER))
else:
    device = torch.device("cpu")

model2 = model2.to(device)

In [ ]:
#loss_func = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model2.parameters(), lr=LR)

In [ ]:
#train_and_validate(model2, loss_func, optimizer, train_dataloader, val_dataloader, device,
#                   epochs=NUM_EPOCHS, batch_size=MINI_BATCH)

In [ ]:
# Usando uma bibliteca com as principais funções
#
# http://www.github.com/dpetrini/nova
#
# Existe um exemplo similar com instruções para rodar completo, neste repositório

from trainer import Trainer

optim_args = {}

train_config = {
    'num_epochs': NUM_EPOCHS,
    'batch_size': MINI_BATCH,
    'name': 'aula',
    'title': 'Cats & Dogs Classifier',
    # 'features': ['auc'],
}

session = Trainer(model2, train_dataloader, val_dataloader, loss_func,
                  optimizer, optim_args, device, train_config)

# train the model
session.train_and_validate()

print("\nRunning models in test set...")
session.run_test(test_dataloader, "normal")
session.run_test(test_dataloader, "best")